In [ ]:
"""
Very rough file for prototyping test logic: the (in development) test suite will need to make sure that the custom sampler is delivering indices in the specified range under the given conditions. 

Critical tests to ensure experimental accuracy include sampling accuracy and data completeness: the custom sampler cannot reduce the amount of data provided to the models due to drop_last settings, for example.
"""

In [2]:
# Close a hole in this notebook server's Dockerfile

import sys
!{sys.executable} -m pip install torchmetrics

     |████████████████████████████████| 274 kB 6.8 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
import os

import torch

from pacsmodeling import (
    PACSDatasetMultipleDomain,
    PACSDatasetSingleDomain,
    PACSSamplerSingleDomainPerBatch
)

In [5]:
pmd = PACSDatasetMultipleDomain(
    "art_painting",
    "train"
)

In [7]:
PS = PACSSamplerSingleDomainPerBatch(
    pmd, 32, False
)

In [11]:
pmd.cumulative_sizes

[2107, 3606, 7137]

In [18]:
"""
This is the logic for a test of the sampler - make sure that each batch comes from exactly one dataset.
"""

for batch_ids in PS:
    lower_bound = 0
    
    for cs in pmd.cumulative_sizes:
        if torch.any(batch_ids < cs) and torch.any(batch_ids > lower_bound):
            print(cs, lower_bound, batch_ids)
            if not torch.all(batch_ids < cs):
                print(f"Failed! One of {batch_ids} not less than {cs}")
            if not torch.all(batch_ids >= lower_bound):
                print(f"Failed! One of {batch_ids} not greater than {lower_bound}")
                
        lower_bound = cs
            

2107 0 tensor([1515,  388,  672,  208, 1479,  291, 1931,   52, 1852, 2096, 1874,  118,
        1609,  940, 1120,  308, 1206,  137,  700,  328, 1059,  757, 1614,  219,
         363, 1092,  522,  733, 2047, 1888,  792,  121])
3606 2107 tensor([2325, 3270, 2524, 3466, 2934, 2326, 2846, 3279, 3516, 2552, 2664, 3206,
        3302, 2579, 3057, 2580, 2314, 2186, 2369, 3512, 2687, 2818, 2743, 3153,
        2436, 2902, 2485, 3577, 2741, 2522, 3301, 2929])
2107 0 tensor([1387, 1735, 1398, 1558, 1133, 1526,  493,  567,  451, 1617, 1554,  380,
        1321,  130,  101, 1520, 1561,  240, 1719,  588, 1641,  394,  302, 1419,
         796,  985,  936,  613, 1131,  267, 2092, 1282])
2107 0 tensor([1603,  834,  149, 1080,  518, 1386, 1677, 1695,  993,  426,  665,  914,
        1022, 1843,  209, 1365,  447,  887, 1032, 1943,  427, 1831,  510, 1806,
        1373,  823, 2040,  139, 1683, 2100,   59,  681])
2107 0 tensor([1851,  997, 1577,  831,  436,  422,  192,  412, 1724, 1680, 1252, 1338,
        1202, 